In [8]:
import pandas
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count

In [9]:
spark = SparkSession.builder \
               .appName('TestMinadzb') \
               .getOrCreate()

In [10]:
csv = spark.read.format("csv").option("header","true").load("dataset.csv")

In [11]:
csv.head()

Row(url='https://bj.lianjia.com/chengjiao/101084782030.html', id='101084782030', Lng='116.475489', Lat='40.01952', Cid='1111027376244', tradeTime='2016-08-09', DOM='1464', followers='106', totalPrice='415.0', price='31680', square='131.0', livingRoom='2', drawingRoom='1', kitchen='1', bathRoom='1', floor='�� 26', buildingType='1', constructionTime='2005', renovationCondition='3', buildingStructure='6', ladderRatio='0.217', elevator='1.0', fiveYearsProperty='0.0', subway='1.0', district='7', communityAverage='56021')

In [12]:
csv.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in csv.columns]
   ).show()

+---+---+---+---+---+---------+------+---------+----------+-----+------+----------+-----------+-------+--------+-----+------------+----------------+-------------------+-----------------+-----------+--------+-----------------+------+--------+----------------+
|url| id|Lng|Lat|Cid|tradeTime|   DOM|followers|totalPrice|price|square|livingRoom|drawingRoom|kitchen|bathRoom|floor|buildingType|constructionTime|renovationCondition|buildingStructure|ladderRatio|elevator|fiveYearsProperty|subway|district|communityAverage|
+---+---+---+---+---+---------+------+---------+----------+-----+------+----------+-----------+-------+--------+-----+------------+----------------+-------------------+-----------------+-----------+--------+-----------------+------+--------+----------------+
|  0|  0|  0|  0|  0|        0|157977|        0|         0|    0|     0|         0|          0|      0|       0|    0|        2021|               0|                  0|                0|          0|      32|               3

In [16]:
csv = csv.drop('url')
csv = csv.drop('cid')
csv = csv.drop('id')
csv = csv.drop('Lng')
csv = csv.drop('Lat')
csv = csv.drop('tradeTime')
csv = csv.drop('floor') #(chinese characters)

csv = csv.drop('DOM') #DOM
csv = csv.drop('price') #price
csv = csv.drop('buildingType') #buildingType
csv = csv.drop('communityAverage') #communityAverage
csv = csv.drop('constructionTime') #constructionTime (chinese characters)

In [17]:
csv.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in csv.columns]
   ).show()

+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+
|followers|totalPrice|square|livingRoom|drawingRoom|kitchen|bathRoom|renovationCondition|buildingStructure|ladderRatio|elevator|fiveYearsProperty|subway|district|
+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+
|        0|         0|     0|         0|          0|      0|       0|                  0|                0|          0|       0|                0|     0|       0|
+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+



In [18]:
csv = csv.dropna()

In [19]:
csv.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in csv.columns]
   ).show()

+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+
|followers|totalPrice|square|livingRoom|drawingRoom|kitchen|bathRoom|renovationCondition|buildingStructure|ladderRatio|elevator|fiveYearsProperty|subway|district|
+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+
|        0|         0|     0|         0|          0|      0|       0|                  0|                0|          0|       0|                0|     0|       0|
+---------+----------+------+----------+-----------+-------+--------+-------------------+-----------------+-----------+--------+-----------------+------+--------+



In [20]:
csv.describe()

DataFrame[summary: string, followers: string, totalPrice: string, square: string, livingRoom: string, drawingRoom: string, kitchen: string, bathRoom: string, renovationCondition: string, buildingStructure: string, ladderRatio: string, elevator: string, fiveYearsProperty: string, subway: string, district: string]

In [21]:
from pyspark.sql.types import DoubleType, IntegerType

to_int = ['followers', 'livingRoom', 'drawingRoom', 'kitchen', 'bathroom', 'renovationCondition', 
          'buildingStructure', 'subway', 'elevator', 'fiveYearsProperty', 'district']
to_double = ['totalPrice', 'ladderRatio', 'square']

for i in to_int:
    csv = csv.withColumn(i,col(i).cast(IntegerType()))
    
for i in to_double:
    csv = csv.withColumn(i,col(i).cast(DoubleType()))

In [22]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=csv.columns, outputCol=vector_col)
df_vector = assembler.transform(csv).select(vector_col)

# get correlation matrix
corr = Correlation.corr(df_vector, vector_col)
corr.collect()[0]["pearson({})".format(vector_col)].values

/usr/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


array([ 1.00000000e+00,  1.52645902e-01, -5.08269143e-02, -6.57602064e-03,
       -5.39928849e-02,  9.10016082e-03, -3.42559962e-02,  2.41063004e-01,
       -2.27037778e-02, -2.00301910e-04, -2.07609856e-02,  4.96317807e-02,
        1.82434638e-02, -3.24042027e-02,  1.52645902e-01,  1.00000000e+00,
        5.75844775e-01,  4.34312367e-01,  3.22706262e-01,  8.64251458e-02,
        4.42432735e-01,  2.50776646e-01,  1.87961612e-01, -1.06371667e-03,
        1.92344785e-01, -3.22497345e-03,  1.13183328e-01,  1.03828020e-02,
       -5.08269143e-02,  5.75844775e-01,  1.00000000e+00,  7.22672921e-01,
        6.16364488e-01,  9.11690035e-02,  7.38126734e-01,  3.14778680e-02,
        1.56220097e-01, -2.42038252e-03,  1.68130705e-01, -1.81643868e-02,
       -9.49727894e-02, -7.33993010e-03, -6.57602064e-03,  4.34312367e-01,
        7.22672921e-01,  1.00000000e+00,  4.89589466e-01,  1.00451606e-01,
        5.52904210e-01,  1.02750923e-02, -6.47624454e-02, -3.25542094e-03,
       -7.82711917e-02,  

In [15]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(
    inputCols = ['followers', 'livingRoom', 'drawingRoom', 'kitchen', 'bathroom', 'renovationCondition', 
          'buildingStructure', 'subway', 'elevator', 'fiveYearsProperty', 'district','ladderRatio', 'square'], 
    outputCol = 'features')
v_df = vectorAssembler.transform(csv)
v_df = v_df.select(['features', 'totalPrice'])
v_df.show(3)

+--------------------+----------+
|            features|totalPrice|
+--------------------+----------+
|[106.0,2.0,1.0,1....|     415.0|
|[126.0,2.0,2.0,1....|     575.0|
|[48.0,3.0,2.0,1.0...|    1030.0|
+--------------------+----------+
only showing top 3 rows



In [16]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [17]:
lr = LinearRegression(featuresCol = 'features', labelCol='totalPrice', maxIter=50, regParam=0.8, elasticNetParam=0.8)
lr_model = lr.fit(train_df)

Coefficients: [0.8776305205712139,19.22917492535458,-13.111185888289869,68.67259918624228,15.101743751650634,34.36825926106769,8.035433449661156,73.83072581435457,12.70982521517611,-10.670378709685108,3.5598343547553193,0.0,3.2356492871272327]
Intercept: -239.18233156854456


In [18]:
trainingSummary = lr_model.summary

#differences between predicted values by the model and the actual values
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)


print("Accuracy: %f" % trainingSummary.r2)

RMSE: 172.600798
Accuracy: 0.441402


In [19]:
train_df.describe().show()

+-------+------------------+
|summary|        totalPrice|
+-------+------------------+
|  count|            222699|
|   mean|348.83457267432897|
| stddev|230.93720343026686|
|    min|               0.1|
|    max|           18130.0|
+-------+------------------+



In [20]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","totalPrice","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="totalPrice",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+----------+--------------------+
|        prediction|totalPrice|            features|
+------------------+----------+--------------------+
| 73.33620319551187|     106.0|(13,[0,1,5,6,10,1...|
| 69.02104894596937|     226.0|(13,[0,1,5,6,10,1...|
|-17.96783062008126|      38.0|(13,[1,5,6,7,10,1...|
| 2.950928959693613|     355.0|(13,[1,5,6,7,10,1...|
|48.769570598871155|     275.0|(13,[1,5,6,8,10,1...|
+------------------+----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.445636


In [21]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 171.556


In [24]:
from pyspark.ml.regression import RandomForestRegressor
rfr = RandomForestRegressor(featuresCol = 'features', labelCol='totalPrice', numTrees=2, maxDepth=2)
rfr_model = rfr.fit(train_df)

In [26]:
train_df.describe().show()

+-------+------------------+
|summary|        totalPrice|
+-------+------------------+
|  count|            222699|
|   mean|348.83457267432897|
| stddev|230.93720343026686|
|    min|               0.1|
|    max|           18130.0|
+-------+------------------+



In [29]:
rfr_predictions = rfr_model.transform(test_df)
rfr_predictions.select("prediction","totalPrice","features").show(15)

rfr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="totalPrice",metricName="r2")

print("R Squared (R2) on test data = %g" % rfr_evaluator.evaluate(rfr_predictions))

+------------------+----------+--------------------+
|        prediction|totalPrice|            features|
+------------------+----------+--------------------+
|252.41567916390704|     106.0|(13,[0,1,5,6,10,1...|
|252.41567916390704|     226.0|(13,[0,1,5,6,10,1...|
|252.41567916390704|      38.0|(13,[1,5,6,7,10,1...|
|252.41567916390704|     355.0|(13,[1,5,6,7,10,1...|
|252.41567916390704|     275.0|(13,[1,5,6,8,10,1...|
|252.41567916390704|      18.5|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|      34.0|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|      35.5|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|      43.8|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|     243.0|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|      38.0|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|     137.0|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|     312.0|[0.0,1.0,0.0,0.0,...|
|306.90358351754924|     453.0|[0.0,1.0,0.0,0.0,...|
|252.41567916390704|      39.5|[0.0,1.0,0.0,0.0,...|
+------------------+----------+---------------

In [31]:
evaluator = RegressionEvaluator(
    labelCol="totalPrice", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 194.172
